In [5]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 137kB/s 
     |████████████████████████████████| 6.8MB 48.7MB/s 
     |████████████████████████████████| 983kB 49.3MB/s 
     |████████████████████████████████| 266kB 48.1MB/s 
     |████████████████████████████████| 1.3MB 51.8MB/s 
     |████████████████████████████████| 1.2MB 43.6MB/s 
     |████████████████████████████████| 471kB 52.8MB/s 
     |████████████████████████████████| 2.9MB 47.9MB/s 
     |████████████████████████████████| 890kB 50.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp37-none-any.whl size=25276443 sha256=6638c537a2d6852c43bdefac5a2306efd23c98c240411fa6c39ac8673899842a
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=a93c7dc8354b37b8a225d82a32ff449e18d1ad7db106ad11c63b4049eb4d3689
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [1]:
import pandas as pd
import numpy as np
import csv
import ktrain

from ktrain import text as classifier_des
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14808007487486874004
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12445326334544554229
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [8]:
from google.colab import files
uploaded = files.upload()

Saving emotion_test.csv to emotion_test (1).csv
Saving emotion_train.csv to emotion_train (1).csv


In [3]:
data_train = pd.read_csv('emotion_train.csv', encoding='utf-8')
data_test = pd.read_csv('emotion_test.csv', encoding='utf-8')

data = data_train.append(data_test, ignore_index=True)
print(data.head())

print('categories: ', data['Emotion'].unique())
print('size of training set: ', (len(data_train['Text'])))
print('size of validation set: ', (len(data_test['Text'])))
print('total data: ', data.Emotion.value_counts())

train_inputs = data_train.Text.tolist()
train_labels = data_test.Text.tolist()

validation_inputs = data_train.Emotion.tolist()
validation_labels = data_test.Emotion.tolist()

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

   Emotion                                               Text
0  neutral   There are tons of other paintings that I thin...
1  sadness  Yet the dog had grown old and less capable , a...
2     fear  When I get into the tube or the train without ...
3     fear  This last may be a source of considerable disq...
4    anger  She disliked the intimacy he showed towards so...
categories:  ['neutral' 'sadness' 'fear' 'anger' 'joy']
size of training set:  7934
size of validation set:  3393
total data:  joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


In [4]:
!pip uninstall keras
!pip install keras --upgrade

Uninstalling Keras-2.4.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras-2.4.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/uninstall.py", line 79, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_install.py", line 755, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_uninstal

In [8]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
validation_inputs = [encoding[x] for x in validation_inputs]
validation_labels = [encoding[x] for x in validation_labels]

In [11]:
(x_train,  y_train), (x_test, y_test), preproc = classifier_des.texts_from_array(x_train=train_inputs, y_train=validation_inputs,
                                                                       x_test=train_labels, y_test=validation_labels,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=500, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [12]:
model = classifier_des.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [13]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [14]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
1323/1323 [==============================] - 1480s 1s/step - loss: 1.2036 - accuracy: 0.4984 - val_loss: 0.5860 - val_accuracy: 0.7893
Epoch 2/3
1323/1323 [==============================] - 1446s 1s/step - loss: 0.4320 - accuracy: 0.8526 - val_loss: 0.5228 - val_accuracy: 0.8246
Epoch 3/3
1323/1323 [==============================] - 1447s 1s/step - loss: 0.1786 - accuracy: 0.9450 - val_loss: 0.5720 - val_accuracy: 0.8240


In [15]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

         joy       0.86      0.84      0.85       707
     sadness       0.82      0.81      0.81       676
        fear       0.87      0.84      0.85       679
       anger       0.78      0.81      0.80       693
     neutral       0.79      0.83      0.81       638

    accuracy                           0.82      3393
   macro avg       0.82      0.82      0.82      3393
weighted avg       0.83      0.82      0.82      3393



array([[593,  17,  16,  18,  63],
       [ 15, 545,  32,  57,  27],
       [ 18,  29, 569,  50,  13],
       [ 16,  51,  30, 562,  34],
       [ 47,  26,   7,  31, 527]])

In [16]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save("base_emote_model")

In [26]:
response = ["I'm doing ok",
            "I'm having a really great day!",
            "Sometimes I wish the ground would consume me",
            "My day was terrible.",
            "Eh, it was alright.",
            "Could be better"]
prediction = predictor.predict(response)

print('predicted: ', prediction)

predicted:  ['neutral', 'joy', 'fear', 'sadness', 'neutral', 'neutral']
